## Kubernetes Clusterwide SSL Demo

### Setup

Create a Kind cluster by using an ansible playbook from the project `ansible` folder.


In [ ]:
!cd ../ansible && ansible-playbook playbooks/kind-cluster.yaml

In [ ]:
!cd ../ansible && ansible-playbook playbooks/setup-ecosystem.yaml

In [1]:
!helm upgrade --install seldon-core-v2-crds  ../k8s/helm-charts/seldon-core-v2-crds -n seldon-mesh

Release "seldon-core-v2-crds" does not exist. Installing it now.
NAME: seldon-core-v2-crds
LAST DEPLOYED: Thu May 11 10:49:42 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [5]:
!kubectl create namespace ns1

namespace/ns1 created


In [51]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns1

NAME: seldon-v2-certs
LAST DEPLOYED: Thu May 11 15:54:24 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [48]:
!helm install seldon-v2 ../k8s/helm-charts/seldon-core-v2-setup/ -n seldon-mesh --set controller.clusterwide=true --values ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

NAME: seldon-v2
LAST DEPLOYED: Thu May 11 15:42:18 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [52]:
!helm install seldon-v2-servers ../k8s/helm-charts/seldon-core-v2-servers  -n ns1 --wait

NAME: seldon-v2-servers
LAST DEPLOYED: Thu May 11 15:55:07 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


### Setup TLS Certiciates for dataplane

In [55]:
MESH_IP=!kubectl get svc seldon-mesh -n ns1 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS1=MESH_IP[0]
import os
os.environ['MESH_IP_NS1'] = MESH_IP_NS1
MESH_IP_NS1

'172.21.255.2'

In [61]:
!seldon config add tls ${PWD}/config-dataplane-tls.json

In [62]:
!seldon config activate tls

### Launch model

In [53]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io/iris created


In [54]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns1

model.mlops.seldon.io/iris condition met


In [63]:
!seldon model infer iris --inference-host ${MESH_IP_NS1}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "6d53cd49-3400-4701-a9d6-25281dc5d2e2",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"data": [
				2
			]
		}
	]
}


In [64]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io "iris" deleted


## TearDown

In [65]:
!helm delete seldon-v2-servers -n ns1

release "seldon-v2-servers" uninstalled
